# Mediapipe

In [1]:
!pip install opencv-python mediapipe


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp

# 얼굴 검출 모델 초기화
mp_face_detector = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

face_detector = mp_face_detector.FaceDetection(model_selection=0, min_detection_confidence=0.5)

I0000 00:00:1747100254.645038  422924 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1747100254.646525  423324 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


### 1. 얼굴(눈,코,입,귀) 인식

In [3]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_detector.process(img_rgb)

    # print(results.detections)

    if results.detections:
        for detection in results.detections:
            mp_drawing.draw_detection(frame, detection)

    cv2.imshow('Face Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

### 2. 얼굴 인식 모자이크

In [4]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = img_rgb.shape

    results = face_detector.process(img_rgb)

    if results.detections:
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box

            x1 = int(max(bbox.xmin, 0) * img_w)
            y1 = int(max(bbox.ymin, 0) * img_h)
            width = int(bbox.width * img_w)
            height = int(bbox.height * img_h)
            x2 = min(x1 + width, img_w)
            y2 = min(y1 + height, img_h)

            face = img_rgb[y1:y2, x1:x2]
            face = cv2.resize(face, (10, 10), interpolation=cv2.INTER_LINEAR)
            face = cv2.resize(face, (x2-x1, y2-y1), interpolation=cv2.INTER_LINEAR)

            img_rgb[y1:y2, x1:x2] = face

    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    cv2.imshow('Face Detection', img_rgb)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

### 3. FaceMesh

In [5]:
import cv2
import mediapipe as mp
import numpy as np

# 얼굴 인식 및 얼굴 메쉬 솔루션 초기화
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

In [6]:
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as face_mesh:

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = face_mesh.process(image)
        image.flags.writeable = True

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_face_landmarks:
            for face_landmark in results.multi_face_landmarks:

                # 얼굴 3D 구조 시각화 (테셀레이션)
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmark,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
                )

                # 얼굴 윤곽선과 주요 특징 강조 (윤곽선, 눈썹, 눈, 입술)
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmark,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
                )

                # 눈의 형태와 위치 강조
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmark,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style()
                )

        cv2.imshow('Face Mesh', image)

        if cv2.waitKey(1) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1747100333.672153  422924 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Pro
W0000 00:00:1747100333.674752  429034 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747100333.682774  429040 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747100333.773529  429034 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


### 4. 필터  씌우기

In [11]:
import cv2
import mediapipe as mp
import numpy as np

mp_draw = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh()

I0000 00:00:1747100405.281247  422924 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Pro
W0000 00:00:1747100405.283678  434026 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


W0000 00:00:1747100405.289167  434026 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


: 

In [8]:
mask_img = cv2.imread('mini.png', cv2.IMREAD_UNCHANGED)

[ WARN:0@112.632] global loadsave.cpp:268 findDecoder imread_('mini.png'): can't open/read file: check file path/integrity


In [9]:
# 얼굴 위에 오버레이 이미지를 합성하는 함수
def face_overlay(background_img, overlay_img, x, y, overlay_size=None):
    try:
        bg_img = background_img.copy()

        if bg_img.shape[2] == 3:
            bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGR2BGRA)

        if overlay_size is not None:
            overlay_img = cv2.resize(overlay_img.copy(), overlay_size)

        b, g, r, a = cv2.split(overlay_img)
        mask = cv2.medianBlur(a, 5)

        h, w, _ = overlay_img.shape
        roi = bg_img[int(y - h/2):int(y + h/2), int(x - w/2):int(x + w/2)]
        img_bg1 = cv2.bitwise_and(roi, roi, mask=cv2.bitwise_not(mask))
        img_bg2 = cv2.bitwise_and(overlay_img.copy(), overlay_img.copy(), mask=mask)
        bg_img[int(y - h/2):int(y + h/2), int(x - w/2):int(x + w/2)] = cv2.add(img_bg1, img_bg2)

        bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGRA2BGR)
        return bg_img

    except:
        return background_img

W0000 00:00:1747100366.815802  431373 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [10]:
cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(img_rgb)
    img_h, img_w, img_c = img.shape

    if result.multi_face_landmarks:
        for face_landmark in result.multi_face_landmarks:
            xy_point = []

            for c, lm in enumerate(face_landmark.landmark):
                xy_point.append([lm.x, lm.y])
                img = cv2.circle(img, (int(lm.x * img_w), int(lm.y * img_h)), 1, (255,0,0), 2)

            top_left = np.min(xy_point, axis=0)
            bottom_right = np.max(xy_point, axis=0)
            mean_xy = np.mean(xy_point, axis=0)
            img = cv2.circle(img, (int(mean_xy[0] * img_w), int(mean_xy[1] * img_h)), 4, (0,0,255), 3)
            face_width = int(bottom_right[0] * img_w) - int(top_left[0] * img_w)
            face_height = int(bottom_right[1] * img_h) - int(top_left[1] * img_h)

            if face_width > 0 and face_height > 0:
                face_result = face_overlay(
                                    img,
                                    mask_img,
                                    int(mean_xy[0] * img_w),
                                    int(mean_xy[1] * img_h),
                                    overlay_size=(face_width, face_height)
                                )
    else:
        face_result = img

    cv2.imshow('mini mouse', face_result)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 